In [1]:
import torch
import datetime
import itertools
import os
from sklearn.metrics import mean_squared_error, roc_auc_score, f1_score

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

print(f"Training on {DEVICE} using PyTorch {torch.__version__}")
print('main3_mlp_federated')

Training on cpu using PyTorch 1.13.1+cpu
main3_mlp_federated


In [3]:
def test(net, x_test, y_test):
  with torch.no_grad():
    y_pred = net(x_test)
    y_pred = y_pred.detach().numpy()
    y_test = y_test.detach().numpy()

    mse = mean_squared_error(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)

    print(auc)
    return mse, auc

In [4]:
#from sklearn.model_selection import train_test_split

X = torch.load('./dataset3/X.pt')
y = torch.load('./dataset3/y.pt')


In [5]:
# Define MLP FL simulation wrapper
from fl_simu import mlp_simulation3

def run_simulation(split, X_train, X_test, y_train, y_test,dir='default', num_epoch=4, batch_size=32, num_rounds=1,
                   layer_size=64, num_of_layers=2, dropout=False):
    return mlp_simulation3(split, X_train, y_train, X_test, y_test, dir, num_epoch=num_epoch, batch_size=batch_size, num_rounds=num_rounds,
                           layer_size=layer_size, num_of_layers=num_of_layers,dropout=dropout)

D:\FD\venv\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [6]:
from sklearn.model_selection import StratifiedKFold

def do_fl(splitting, num_rounds_list, num_epochs_list = [1], batch_size_list = [32], dropout_list = [False],X=X, y=y):
  start = datetime.datetime.now()
  mses = []
  aucs = []

  kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

  test_name = 'Dataset 3 MLP'
  directory_path = 'model_checkpoints3/' + test_name

  if not os.path.exists(directory_path):
    os.makedirs(directory_path)


  flag = 1

  for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Fold {fold_idx + 1}:")

    if flag == 2:
      break

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    X_train = torch.tensor(X_train)
    X_test = torch.tensor(X_test)
    y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
    y_test = torch.reshape(torch.tensor(y_test), (-1, 1))

    X_test = X_test.to(DEVICE)
    y_test = y_test.to(DEVICE)

    print(f'x_test: {X_test.device}')
    print(f'y_test: {y_test.device}')

    for batch_size, num_epoch, num_rounds, dropout in itertools.product(
            batch_size_list,
            num_epochs_list,
            num_rounds_list,
            dropout_list):

      print(num_rounds)

      model_fl = run_simulation(splitting,X_train, X_test, y_train, y_test, test_name, batch_size=batch_size, num_epoch=num_epoch, num_rounds=num_rounds,dropout=dropout)

      model_fl.cpu()

      X_test = X_test.cpu()
      y_test = y_test.cpu()

      mse, auc= test(model_fl, X_test, y_test)
      mses.append(mse)
      aucs.append(auc)

      flag += 1

  end = datetime.datetime.now()
  time = end - start

  print('splitting', splitting)
  print('time', time)
  print('mse', mses)
  print('auc', aucs)


In [7]:
num_rounds = [30] # 10

splitting = [0.5, 0.5]
do_fl(splitting, num_rounds)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Fold 1:
x_test: cpu
y_test: cpu
30
Client 0 trainset shape torch.Size([111896, 10])
Client 1 trainset shape torch.Size([112064, 10])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 22:03:54,844	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8269 
INFO flwr 2023-08-31 22:04:03,244 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 830396007.0, 'object_store_memory': 415198003.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 22:04:03,246 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 22:04:03,247 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 22:04:12,392 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 22:04:12,394 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 22:04:12,396 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 22:04:12,397 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 22:04:27,301 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 22:04:27,308 | fedavg.p

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-31 22:04:27,511 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 2)


...

agrregate_evaluate results list length is  2
server round is  29
loss is  0.1534539759159088


DEBUG flwr 2023-08-31 22:11:53,555 | server.py:236 | fit_round 30 received 2 results and 0 failures
DEBUG flwr 2023-08-31 22:11:53,579 | server.py:173 | evaluate_round 30: strategy sampled 2 clients (out of 2)


agrregate_fit results list length is  2
Saving round 30 aggregated_parameters...


DEBUG flwr 2023-08-31 22:11:53,782 | server.py:187 | evaluate_round 30 received 2 results and 0 failures
INFO flwr 2023-08-31 22:11:53,784 | server.py:153 | FL finished in 461.3873453999986
INFO flwr 2023-08-31 22:11:53,803 | app.py:225 | app_fit: losses_distributed [(1, 0.25696372985839844), (2, 0.2175365388393402), (3, 0.18803240358829498), (4, 0.16692674160003662), (5, 0.16508275270462036), (6, 0.16275936365127563), (7, 0.15826311707496643), (8, 0.15889957547187805), (9, 0.1585818976163864), (10, 0.15910056233406067), (11, 0.1567535102367401), (12, 0.1584070324897766), (13, 0.15833550691604614), (14, 0.15850940346717834), (15, 0.15732905268669128), (16, 0.15875300765037537), (17, 0.15590941905975342), (18, 0.15605592727661133), (19, 0.15696276724338531), (20, 0.15618161857128143), (21, 0.15905636548995972), (22, 0.1574641913175583), (23, 0.15434671938419342), (24, 0.1547364741563797), (25, 0.15609672665596008), (26, 0.15796834230422974), (27, 0.153464674949646), (28, 0.1515746861696

agrregate_evaluate results list length is  2
server round is  30
loss is  0.15249015390872955
History (loss, distributed):
	round 1: 0.25696372985839844
	round 2: 0.2175365388393402
	round 3: 0.18803240358829498
	round 4: 0.16692674160003662
	round 5: 0.16508275270462036
	round 6: 0.16275936365127563
	round 7: 0.15826311707496643
	round 8: 0.15889957547187805
	round 9: 0.1585818976163864
	round 10: 0.15910056233406067
	round 11: 0.1567535102367401
	round 12: 0.1584070324897766
	round 13: 0.15833550691604614
	round 14: 0.15850940346717834
	round 15: 0.15732905268669128
	round 16: 0.15875300765037537
	round 17: 0.15590941905975342
	round 18: 0.15605592727661133
	round 19: 0.15696276724338531
	round 20: 0.15618161857128143
	round 21: 0.15905636548995972
	round 22: 0.1574641913175583
	round 23: 0.15434671938419342
	round 24: 0.1547364741563797
	round 25: 0.15609672665596008
	round 26: 0.15796834230422974
	round 27: 0.153464674949646
	round 28: 0.15157468616962433
	round 29: 0.1534539759159

In [9]:

splitting = [0.6, 0.4]
do_fl(splitting, num_rounds)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Fold 1:
x_test: cpu
y_test: cpu
30
Client 0 trainset shape torch.Size([134296, 10])
Client 1 trainset shape torch.Size([89664, 10])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 22:14:37,828	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 
INFO flwr 2023-08-31 22:14:58,192 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 621057639.0, 'object_store_memory': 310528819.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 22:14:58,195 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 22:14:58,198 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 22:15:11,590 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 22:15:11,594 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 22:15:11,596 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 22:15:11,598 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 22:15:48,677 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 22:15:48,687 | fedavg.p

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-31 22:15:48,927 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2023-08-31 22:15:48,929 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 2)


...

agrregate_fit results list length is  2
Saving round 29 aggregated_parameters...


DEBUG flwr 2023-08-31 22:23:40,372 | server.py:222 | fit_round 30: strategy sampled 2 clients (out of 2)


agrregate_evaluate results list length is  2
server round is  29
loss is  0.15093988180160522


DEBUG flwr 2023-08-31 22:23:54,088 | server.py:236 | fit_round 30 received 2 results and 0 failures
DEBUG flwr 2023-08-31 22:23:54,101 | server.py:173 | evaluate_round 30: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 22:23:54,263 | server.py:187 | evaluate_round 30 received 2 results and 0 failures
INFO flwr 2023-08-31 22:23:54,265 | server.py:153 | FL finished in 522.6665451000154
INFO flwr 2023-08-31 22:23:54,266 | app.py:225 | app_fit: losses_distributed [(1, 0.23020093142986298), (2, 0.2097826451063156), (3, 0.18446379899978638), (4, 0.16906185448169708), (5, 0.16247044503688812), (6, 0.16115622222423553), (7, 0.16090771555900574), (8, 0.15916725993156433), (9, 0.1540849357843399), (10, 0.15354406833648682), (11, 0.15608319640159607), (12, 0.15983593463897705), (13, 0.1546599566936493), (14, 0.15447261929512024), (15, 0.1539773792028427), (16, 0.15554751455783844), (17, 0.1544819325208664), (18, 0.15586881339550018), (19, 0.15286920964717865), (20, 0.1534519046545028

agrregate_fit results list length is  2
Saving round 30 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  30
loss is  0.1539747416973114
History (loss, distributed):
	round 1: 0.23020093142986298
	round 2: 0.2097826451063156
	round 3: 0.18446379899978638
	round 4: 0.16906185448169708
	round 5: 0.16247044503688812
	round 6: 0.16115622222423553
	round 7: 0.16090771555900574
	round 8: 0.15916725993156433
	round 9: 0.1540849357843399
	round 10: 0.15354406833648682
	round 11: 0.15608319640159607
	round 12: 0.15983593463897705
	round 13: 0.1546599566936493
	round 14: 0.15447261929512024
	round 15: 0.1539773792028427
	round 16: 0.15554751455783844
	round 17: 0.1544819325208664
	round 18: 0.15586881339550018
	round 19: 0.15286920964717865
	round 20: 0.15345190465450287
	round 21: 0.15510699152946472
	round 22: 0.15209248661994934
	round 23: 0.15184147655963898
	round 24: 0.15157103538513184
	round 25: 0.1547592729330063
	round 26: 0.15431201457977295
	round

In [10]:

splitting = [0.8, 0.2]
do_fl(splitting, num_rounds)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Fold 1:
x_test: cpu
y_test: cpu
30
Client 0 trainset shape torch.Size([179214, 10])
Client 1 trainset shape torch.Size([44746, 10])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 22:30:29,791	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 
INFO flwr 2023-08-31 22:30:40,233 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1167701607.0, 'object_store_memory': 583850803.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 22:30:40,236 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 22:30:40,237 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 22:30:48,916 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 22:30:48,918 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 22:30:48,919 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 22:30:48,920 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 22:31:08,890 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 22:31:08,897 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  0.2322271764278412


DEBUG flwr 2023-08-31 22:31:23,924 | server.py:236 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 22:31:23,938 | server.py:173 | evaluate_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 22:31:24,078 | server.py:187 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 22:31:24,079 | server.py:222 | fit_round 3: strategy sampled 2 clients (out of 2)


...

agrregate_evaluate results list length is  2
server round is  29
loss is  0.15247687697410583


DEBUG flwr 2023-08-31 22:44:07,074 | server.py:236 | fit_round 30 received 2 results and 0 failures
DEBUG flwr 2023-08-31 22:44:07,101 | server.py:173 | evaluate_round 30: strategy sampled 2 clients (out of 2)


agrregate_fit results list length is  2
Saving round 30 aggregated_parameters...


DEBUG flwr 2023-08-31 22:44:07,288 | server.py:187 | evaluate_round 30 received 2 results and 0 failures
INFO flwr 2023-08-31 22:44:07,290 | server.py:153 | FL finished in 798.3699418999895
INFO flwr 2023-08-31 22:44:07,291 | app.py:225 | app_fit: losses_distributed [(1, 0.2322271764278412), (2, 0.20108048617839813), (3, 0.1751890927553177), (4, 0.15939538180828094), (5, 0.1560927927494049), (6, 0.15553855895996094), (7, 0.15609849989414215), (8, 0.15707167983055115), (9, 0.15703584253787994), (10, 0.15635228157043457), (11, 0.151996910572052), (12, 0.1577184796333313), (13, 0.15171150863170624), (14, 0.1509125828742981), (15, 0.15385355055332184), (16, 0.15512649714946747), (17, 0.154957115650177), (18, 0.15225331485271454), (19, 0.1553984433412552), (20, 0.15398746728897095), (21, 0.15264040231704712), (22, 0.1533135026693344), (23, 0.15576104819774628), (24, 0.15550389885902405), (25, 0.14904753863811493), (26, 0.15310132503509521), (27, 0.15067802369594574), (28, 0.1500249058008194

agrregate_evaluate results list length is  2
server round is  30
loss is  0.15011975169181824
History (loss, distributed):
	round 1: 0.2322271764278412
	round 2: 0.20108048617839813
	round 3: 0.1751890927553177
	round 4: 0.15939538180828094
	round 5: 0.1560927927494049
	round 6: 0.15553855895996094
	round 7: 0.15609849989414215
	round 8: 0.15707167983055115
	round 9: 0.15703584253787994
	round 10: 0.15635228157043457
	round 11: 0.151996910572052
	round 12: 0.1577184796333313
	round 13: 0.15171150863170624
	round 14: 0.1509125828742981
	round 15: 0.15385355055332184
	round 16: 0.15512649714946747
	round 17: 0.154957115650177
	round 18: 0.15225331485271454
	round 19: 0.1553984433412552
	round 20: 0.15398746728897095
	round 21: 0.15264040231704712
	round 22: 0.1533135026693344
	round 23: 0.15576104819774628
	round 24: 0.15550389885902405
	round 25: 0.14904753863811493
	round 26: 0.15310132503509521
	round 27: 0.15067802369594574
	round 28: 0.1500249058008194
	round 29: 0.15247687697410583

In [11]:
# num_rounds = [15]
#
splitting = [0.34, 0.33, 0.33]
do_fl(splitting, num_rounds)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Fold 1:
x_test: cpu
y_test: cpu
30
Client 0 trainset shape torch.Size([76102, 10])
Client 1 trainset shape torch.Size([71590, 10])
Client 2 trainset shape torch.Size([71764, 10])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 22:44:20,367	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 
INFO flwr 2023-08-31 22:44:31,441 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1210195968.0, 'object_store_memory': 605097984.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 22:44:31,445 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 22:44:31,447 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 22:44:37,042 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 22:44:37,043 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 22:44:37,044 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 22:44:37,045 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 22:44:53,151 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 22:44:53,161 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-31 22:44:53,383 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2023-08-31 22:44:53,385 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 3)


...

agrregate_evaluate results list length is  2
server round is  29
loss is  0.15783073008060455


DEBUG flwr 2023-08-31 22:51:20,400 | server.py:236 | fit_round 30 received 2 results and 0 failures
DEBUG flwr 2023-08-31 22:51:20,426 | server.py:173 | evaluate_round 30: strategy sampled 2 clients (out of 3)


agrregate_fit results list length is  2
Saving round 30 aggregated_parameters...


DEBUG flwr 2023-08-31 22:51:20,705 | server.py:187 | evaluate_round 30 received 2 results and 0 failures
INFO flwr 2023-08-31 22:51:20,707 | server.py:153 | FL finished in 403.66309550000005
INFO flwr 2023-08-31 22:51:20,708 | app.py:225 | app_fit: losses_distributed [(1, 0.2588854730129242), (2, 0.2384224683046341), (3, 0.22131051123142242), (4, 0.20510053634643555), (5, 0.1867547631263733), (6, 0.17268085479736328), (7, 0.1675434410572052), (8, 0.1594303846359253), (9, 0.16640695929527283), (10, 0.1612403690814972), (11, 0.15690937638282776), (12, 0.15295280516147614), (13, 0.1547263115644455), (14, 0.15328307449817657), (15, 0.1597450077533722), (16, 0.15594923496246338), (17, 0.15787489712238312), (18, 0.15765559673309326), (19, 0.16034583747386932), (20, 0.15215139091014862), (21, 0.15781041979789734), (22, 0.15326842665672302), (23, 0.1556898057460785), (24, 0.153313547372818), (25, 0.1541467010974884), (26, 0.15714582800865173), (27, 0.15353839099407196), (28, 0.1533342599868774

agrregate_evaluate results list length is  2
server round is  30
loss is  0.15758877992630005
History (loss, distributed):
	round 1: 0.2588854730129242
	round 2: 0.2384224683046341
	round 3: 0.22131051123142242
	round 4: 0.20510053634643555
	round 5: 0.1867547631263733
	round 6: 0.17268085479736328
	round 7: 0.1675434410572052
	round 8: 0.1594303846359253
	round 9: 0.16640695929527283
	round 10: 0.1612403690814972
	round 11: 0.15690937638282776
	round 12: 0.15295280516147614
	round 13: 0.1547263115644455
	round 14: 0.15328307449817657
	round 15: 0.1597450077533722
	round 16: 0.15594923496246338
	round 17: 0.15787489712238312
	round 18: 0.15765559673309326
	round 19: 0.16034583747386932
	round 20: 0.15215139091014862
	round 21: 0.15781041979789734
	round 22: 0.15326842665672302
	round 23: 0.1556898057460785
	round 24: 0.153313547372818
	round 25: 0.1541467010974884
	round 26: 0.15714582800865173
	round 27: 0.15353839099407196
	round 28: 0.15333425998687744
	round 29: 0.15783073008060455

In [12]:
splitting = [0.6, 0.2, 0.2]
do_fl(splitting, num_rounds)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Fold 1:
x_test: cpu
y_test: cpu
30
Client 0 trainset shape torch.Size([134560, 10])
Client 1 trainset shape torch.Size([44706, 10])
Client 2 trainset shape torch.Size([44694, 10])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 23:06:30,204	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 
INFO flwr 2023-08-31 23:06:42,446 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1236450510.0, 'object_store_memory': 618225254.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 23:06:42,450 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 23:06:42,453 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 23:06:47,890 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 23:06:47,892 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 23:06:47,894 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 23:06:47,896 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 23:07:00,014 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 23:07:00,025 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-31 23:07:00,636 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2023-08-31 23:07:00,639 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 3)


...

agrregate_evaluate results list length is  2
server round is  28
loss is  0.1523573398590088


DEBUG flwr 2023-08-31 23:12:16,878 | server.py:236 | fit_round 29 received 2 results and 0 failures
DEBUG flwr 2023-08-31 23:12:16,893 | server.py:173 | evaluate_round 29: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 23:12:17,053 | server.py:187 | evaluate_round 29 received 2 results and 0 failures
DEBUG flwr 2023-08-31 23:12:17,054 | server.py:222 | fit_round 30: strategy sampled 2 clients (out of 3)


agrregate_fit results list length is  2
Saving round 29 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  29
loss is  0.15473905205726624


DEBUG flwr 2023-08-31 23:12:29,359 | server.py:236 | fit_round 30 received 2 results and 0 failures
DEBUG flwr 2023-08-31 23:12:29,371 | server.py:173 | evaluate_round 30: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 23:12:29,524 | server.py:187 | evaluate_round 30 received 2 results and 0 failures
INFO flwr 2023-08-31 23:12:29,525 | server.py:153 | FL finished in 341.6292247999809
INFO flwr 2023-08-31 23:12:29,526 | app.py:225 | app_fit: losses_distributed [(1, 0.22972756624221802), (2, 0.21736463904380798), (3, 0.1894512176513672), (4, 0.17257700860500336), (5, 0.16185404360294342), (6, 0.1616530418395996), (7, 0.1617799699306488), (8, 0.15744265913963318), (9, 0.16115237772464752), (10, 0.15916600823402405), (11, 0.15671899914741516), (12, 0.15876513719558716), (13, 0.1522688865661621), (14, 0.15219153463840485), (15, 0.1548018902540207), (16, 0.15936413407325745), (17, 0.1522296518087387), (18, 0.15975983440876007), (19, 0.15077784657478333), (20, 0.1554877907037735)

agrregate_fit results list length is  2
Saving round 30 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  30
loss is  0.1506582349538803
History (loss, distributed):
	round 1: 0.22972756624221802
	round 2: 0.21736463904380798
	round 3: 0.1894512176513672
	round 4: 0.17257700860500336
	round 5: 0.16185404360294342
	round 6: 0.1616530418395996
	round 7: 0.1617799699306488
	round 8: 0.15744265913963318
	round 9: 0.16115237772464752
	round 10: 0.15916600823402405
	round 11: 0.15671899914741516
	round 12: 0.15876513719558716
	round 13: 0.1522688865661621
	round 14: 0.15219153463840485
	round 15: 0.1548018902540207
	round 16: 0.15936413407325745
	round 17: 0.1522296518087387
	round 18: 0.15975983440876007
	round 19: 0.15077784657478333
	round 20: 0.1554877907037735
	round 21: 0.15580305457115173
	round 22: 0.1514086276292801
	round 23: 0.15479691326618195
	round 24: 0.15527936816215515
	round 25: 0.15731364488601685
	round 26: 0.15348026156425476
	round 2

In [13]:
splitting = [0.8, 0.1, 0.1]
do_fl(splitting, num_rounds)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Fold 1:
x_test: cpu
y_test: cpu
30
Client 0 trainset shape torch.Size([179090, 10])
Client 1 trainset shape torch.Size([22416, 10])
Client 2 trainset shape torch.Size([22454, 10])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 23:12:39,444	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 
INFO flwr 2023-08-31 23:12:47,900 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1540534272.0, 'object_store_memory': 770267136.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 23:12:47,902 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 23:12:47,903 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 23:12:51,985 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 23:12:51,986 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 23:12:51,987 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 23:12:51,988 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 23:13:08,521 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 23:13:08,527 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-31 23:13:08,749 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2023-08-31 23:13:08,750 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 3)


...

agrregate_evaluate results list length is  2
server round is  29
loss is  0.15941846370697021


DEBUG flwr 2023-08-31 23:18:43,485 | server.py:236 | fit_round 30 received 2 results and 0 failures
DEBUG flwr 2023-08-31 23:18:43,507 | server.py:173 | evaluate_round 30: strategy sampled 2 clients (out of 3)


agrregate_fit results list length is  2
Saving round 30 aggregated_parameters...


DEBUG flwr 2023-08-31 23:18:43,784 | server.py:187 | evaluate_round 30 received 2 results and 0 failures
INFO flwr 2023-08-31 23:18:43,786 | server.py:153 | FL finished in 351.7991303000017
INFO flwr 2023-08-31 23:18:43,787 | app.py:225 | app_fit: losses_distributed [(1, 0.22937068343162537), (2, 0.19122201204299927), (3, 0.16792094707489014), (4, 0.15782392024993896), (5, 0.1586078703403473), (6, 0.15270578861236572), (7, 0.15479448437690735), (8, 0.1515873372554779), (9, 0.14987459778785706), (10, 0.15579670667648315), (11, 0.1537591516971588), (12, 0.1515982747077942), (13, 0.15848490595817566), (14, 0.16070306301116943), (15, 0.15628786385059357), (16, 0.15616920590400696), (17, 0.1561053842306137), (18, 0.1538374125957489), (19, 0.1573977768421173), (20, 0.15895946323871613), (21, 0.15492312610149384), (22, 0.15423624217510223), (23, 0.15715311467647552), (24, 0.15360049903392792), (25, 0.15567931532859802), (26, 0.15116584300994873), (27, 0.1579568088054657), (28, 0.1534701138734

agrregate_evaluate results list length is  2
server round is  30
loss is  0.15358026325702667
History (loss, distributed):
	round 1: 0.22937068343162537
	round 2: 0.19122201204299927
	round 3: 0.16792094707489014
	round 4: 0.15782392024993896
	round 5: 0.1586078703403473
	round 6: 0.15270578861236572
	round 7: 0.15479448437690735
	round 8: 0.1515873372554779
	round 9: 0.14987459778785706
	round 10: 0.15579670667648315
	round 11: 0.1537591516971588
	round 12: 0.1515982747077942
	round 13: 0.15848490595817566
	round 14: 0.16070306301116943
	round 15: 0.15628786385059357
	round 16: 0.15616920590400696
	round 17: 0.1561053842306137
	round 18: 0.1538374125957489
	round 19: 0.1573977768421173
	round 20: 0.15895946323871613
	round 21: 0.15492312610149384
	round 22: 0.15423624217510223
	round 23: 0.15715311467647552
	round 24: 0.15360049903392792
	round 25: 0.15567931532859802
	round 26: 0.15116584300994873
	round 27: 0.1579568088054657
	round 28: 0.15347011387348175
	round 29: 0.1594184637069

In [14]:
#num_rounds = [20]

splitting = [0.2, 0.2, 0.2, 0.2, 0.2]
do_fl(splitting, num_rounds)

Fold 1:
x_test: cpu
y_test: cpu
30


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Client 0 trainset shape torch.Size([44824, 10])
Client 1 trainset shape torch.Size([44744, 10])
Client 2 trainset shape torch.Size([44706, 10])
Client 3 trainset shape torch.Size([44880, 10])
Client 4 trainset shape torch.Size([44806, 10])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 23:18:54,546	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 
INFO flwr 2023-08-31 23:19:03,432 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1427627214.0, 'object_store_memory': 713813606.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 23:19:03,436 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 23:19:03,437 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 23:19:08,997 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 23:19:08,999 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 23:19:09,000 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 23:19:09,000 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 23:19:17,777 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 23:19:17,783 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  0.23610715568065643


...

agrregate_evaluate results list length is  2
server round is  29
loss is  0.15505851805210114


DEBUG flwr 2023-08-31 23:21:50,494 | server.py:236 | fit_round 30 received 2 results and 0 failures
DEBUG flwr 2023-08-31 23:21:50,514 | server.py:173 | evaluate_round 30: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 23:21:50,684 | server.py:187 | evaluate_round 30 received 2 results and 0 failures


agrregate_fit results list length is  2
Saving round 30 aggregated_parameters...


INFO flwr 2023-08-31 23:21:50,687 | server.py:153 | FL finished in 161.68621109999367
INFO flwr 2023-08-31 23:21:50,688 | app.py:225 | app_fit: losses_distributed [(1, 0.23610715568065643), (2, 0.23938621580600739), (3, 0.23435063660144806), (4, 0.22675293684005737), (5, 0.2157822549343109), (6, 0.20567916333675385), (7, 0.19309915602207184), (8, 0.18488037586212158), (9, 0.17953220009803772), (10, 0.17318302392959595), (11, 0.16954785585403442), (12, 0.1691139042377472), (13, 0.16016386449337006), (14, 0.1627955138683319), (15, 0.1598539501428604), (16, 0.16292668879032135), (17, 0.16295260190963745), (18, 0.15779587626457214), (19, 0.1556301712989807), (20, 0.15430064499378204), (21, 0.15792390704154968), (22, 0.15663598477840424), (23, 0.15691666305065155), (24, 0.15466119349002838), (25, 0.15625829994678497), (26, 0.15677234530448914), (27, 0.1560976356267929), (28, 0.15392620861530304), (29, 0.15505851805210114), (30, 0.15901128947734833)]
INFO flwr 2023-08-31 23:21:50,689 | app.p

agrregate_evaluate results list length is  2
server round is  30
loss is  0.15901128947734833
History (loss, distributed):
	round 1: 0.23610715568065643
	round 2: 0.23938621580600739
	round 3: 0.23435063660144806
	round 4: 0.22675293684005737
	round 5: 0.2157822549343109
	round 6: 0.20567916333675385
	round 7: 0.19309915602207184
	round 8: 0.18488037586212158
	round 9: 0.17953220009803772
	round 10: 0.17318302392959595
	round 11: 0.16954785585403442
	round 12: 0.1691139042377472
	round 13: 0.16016386449337006
	round 14: 0.1627955138683319
	round 15: 0.1598539501428604
	round 16: 0.16292668879032135
	round 17: 0.16295260190963745
	round 18: 0.15779587626457214
	round 19: 0.1556301712989807
	round 20: 0.15430064499378204
	round 21: 0.15792390704154968
	round 22: 0.15663598477840424
	round 23: 0.15691666305065155
	round 24: 0.15466119349002838
	round 25: 0.15625829994678497
	round 26: 0.15677234530448914
	round 27: 0.1560976356267929
	round 28: 0.15392620861530304
	round 29: 0.15505851805

In [15]:
splitting = [0.6, 0.1, 0.1, 0.1, 0.1]
do_fl(splitting, num_rounds)


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Fold 1:
x_test: cpu
y_test: cpu
30
Client 0 trainset shape torch.Size([134384, 10])
Client 1 trainset shape torch.Size([22368, 10])
Client 2 trainset shape torch.Size([22380, 10])
Client 3 trainset shape torch.Size([22344, 10])
Client 4 trainset shape torch.Size([22484, 10])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 23:34:39,831	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 
INFO flwr 2023-08-31 23:34:54,621 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1617567744.0, 'object_store_memory': 808783872.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 23:34:54,625 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 23:34:54,626 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 23:35:02,413 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 23:35:02,414 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 23:35:02,415 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 23:35:02,416 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 23:35:34,732 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 23:35:34,751 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-31 23:35:35,156 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2023-08-31 23:35:35,159 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 5)


...

agrregate_evaluate results list length is  2
server round is  29
loss is  0.1549636423587799


DEBUG flwr 2023-08-31 23:40:11,144 | server.py:236 | fit_round 30 received 2 results and 0 failures
DEBUG flwr 2023-08-31 23:40:11,158 | server.py:173 | evaluate_round 30: strategy sampled 2 clients (out of 5)


agrregate_fit results list length is  2
Saving round 30 aggregated_parameters...


DEBUG flwr 2023-08-31 23:40:11,357 | server.py:187 | evaluate_round 30 received 2 results and 0 failures
INFO flwr 2023-08-31 23:40:11,358 | server.py:153 | FL finished in 308.9421450999798
INFO flwr 2023-08-31 23:40:11,359 | app.py:225 | app_fit: losses_distributed [(1, 0.2376198023557663), (2, 0.23401542007923126), (3, 0.22991377115249634), (4, 0.22439375519752502), (5, 0.19765974581241608), (6, 0.17478539049625397), (7, 0.16519959270954132), (8, 0.16638022661209106), (9, 0.1701480746269226), (10, 0.16052773594856262), (11, 0.158692866563797), (12, 0.15615199506282806), (13, 0.15814515948295593), (14, 0.15874122083187103), (15, 0.15466447174549103), (16, 0.15985693037509918), (17, 0.15523909032344818), (18, 0.1531909555196762), (19, 0.15972329676151276), (20, 0.15589378774166107), (21, 0.15331345796585083), (22, 0.15735302865505219), (23, 0.1540968269109726), (24, 0.14838550984859467), (25, 0.15201100707054138), (26, 0.15629072487354279), (27, 0.15152877569198608), (28, 0.15735697746

agrregate_evaluate results list length is  2
server round is  30
loss is  0.15353956818580627
History (loss, distributed):
	round 1: 0.2376198023557663
	round 2: 0.23401542007923126
	round 3: 0.22991377115249634
	round 4: 0.22439375519752502
	round 5: 0.19765974581241608
	round 6: 0.17478539049625397
	round 7: 0.16519959270954132
	round 8: 0.16638022661209106
	round 9: 0.1701480746269226
	round 10: 0.16052773594856262
	round 11: 0.158692866563797
	round 12: 0.15615199506282806
	round 13: 0.15814515948295593
	round 14: 0.15874122083187103
	round 15: 0.15466447174549103
	round 16: 0.15985693037509918
	round 17: 0.15523909032344818
	round 18: 0.1531909555196762
	round 19: 0.15972329676151276
	round 20: 0.15589378774166107
	round 21: 0.15331345796585083
	round 22: 0.15735302865505219
	round 23: 0.1540968269109726
	round 24: 0.14838550984859467
	round 25: 0.15201100707054138
	round 26: 0.15629072487354279
	round 27: 0.15152877569198608
	round 28: 0.15735697746276855
	round 29: 0.15496364235

In [16]:

splitting = [0.8, 0.05, 0.05, 0.05, 0.05]
do_fl(splitting, num_rounds)

Fold 1:
x_test: cpu
y_test: cpu
30
Client 0 trainset shape torch.Size([179088, 10])
Client 1 trainset shape torch.Size([8964, 10])
Client 2 trainset shape torch.Size([9004, 10])
Client 3 trainset shape torch.Size([8958, 10])


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Client 4 trainset shape torch.Size([8978, 10])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 23:40:22,156	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8270 
INFO flwr 2023-08-31 23:40:32,429 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1160390247.0, 'object_store_memory': 580195123.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 23:40:32,431 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 23:40:32,433 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 23:40:38,721 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 23:40:38,721 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 23:40:38,722 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 23:40:38,723 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 23:40:43,203 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 23:40:43,209 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  0.22440436482429504


...

agrregate_evaluate results list length is  2
server round is  29
loss is  0.1520221084356308


DEBUG flwr 2023-08-31 23:44:59,392 | server.py:236 | fit_round 30 received 2 results and 0 failures
DEBUG flwr 2023-08-31 23:44:59,427 | server.py:173 | evaluate_round 30: strategy sampled 2 clients (out of 5)


agrregate_fit results list length is  2
Saving round 30 aggregated_parameters...


DEBUG flwr 2023-08-31 23:44:59,647 | server.py:187 | evaluate_round 30 received 2 results and 0 failures
INFO flwr 2023-08-31 23:44:59,649 | server.py:153 | FL finished in 260.9259936999879
INFO flwr 2023-08-31 23:44:59,651 | app.py:225 | app_fit: losses_distributed [(1, 0.22440436482429504), (2, 0.213456928730011), (3, 0.18206356465816498), (4, 0.18429973721504211), (5, 0.18271611630916595), (6, 0.18283545970916748), (7, 0.17752499878406525), (8, 0.1817624568939209), (9, 0.16586193442344666), (10, 0.16186222434043884), (11, 0.15976673364639282), (12, 0.15597034990787506), (13, 0.15509769320487976), (14, 0.15846426784992218), (15, 0.15833762288093567), (16, 0.15692438185214996), (17, 0.15419702231884003), (18, 0.16111810505390167), (19, 0.1562335342168808), (20, 0.16069044172763824), (21, 0.15420077741146088), (22, 0.15637534856796265), (23, 0.1502046436071396), (24, 0.15543627738952637), (25, 0.1527136266231537), (26, 0.14998316764831543), (27, 0.15798552334308624), (28, 0.15145996212

agrregate_evaluate results list length is  2
server round is  30
loss is  0.15270750224590302
History (loss, distributed):
	round 1: 0.22440436482429504
	round 2: 0.213456928730011
	round 3: 0.18206356465816498
	round 4: 0.18429973721504211
	round 5: 0.18271611630916595
	round 6: 0.18283545970916748
	round 7: 0.17752499878406525
	round 8: 0.1817624568939209
	round 9: 0.16586193442344666
	round 10: 0.16186222434043884
	round 11: 0.15976673364639282
	round 12: 0.15597034990787506
	round 13: 0.15509769320487976
	round 14: 0.15846426784992218
	round 15: 0.15833762288093567
	round 16: 0.15692438185214996
	round 17: 0.15419702231884003
	round 18: 0.16111810505390167
	round 19: 0.1562335342168808
	round 20: 0.16069044172763824
	round 21: 0.15420077741146088
	round 22: 0.15637534856796265
	round 23: 0.1502046436071396
	round 24: 0.15543627738952637
	round 25: 0.1527136266231537
	round 26: 0.14998316764831543
	round 27: 0.15798552334308624
	round 28: 0.1514599621295929
	round 29: 0.152022108435

In [18]:
#num_rounds = [30]

splitting = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
do_fl(splitting, num_rounds)

C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Fold 1:
x_test: cpu
y_test: cpu
30
Client 0 trainset shape torch.Size([22394, 10])
Client 1 trainset shape torch.Size([22412, 10])
Client 2 trainset shape torch.Size([22340, 10])
Client 3 trainset shape torch.Size([22350, 10])
Client 4 trainset shape torch.Size([22356, 10])
Client 5 trainset shape torch.Size([22428, 10])
Client 6 trainset shape torch.Size([22422, 10])
Client 7 trainset shape torch.Size([22396, 10])
Client 8 trainset shape torch.Size([22466, 10])
Client 9 trainset shape torch.Size([22396, 10])
FL started training on cpu
Number of available GPUs: 0


2023-09-01 00:02:16,526	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8271 
INFO flwr 2023-09-01 00:02:27,033 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1282378139.0, 'object_store_memory': 641189068.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-01 00:02:27,036 | server.py:89 | Initializing global parameters
INFO flwr 2023-09-01 00:02:27,038 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-09-01 00:02:36,502 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-09-01 00:02:36,503 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-09-01 00:02:36,504 | server.py:104 | FL starting
DEBUG flwr 2023-09-01 00:02:36,505 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-09-01 00:02:50,096 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-09-01 00:02:50,129 | fedavg

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-09-01 00:02:50,589 | server.py:187 | evaluate_round 1 received 5 results and 0 failures
DEBUG flwr 2023-09-01 00:02:50,590 | server.py:222 | fit_round 2: strategy sampled 5 clients (out of 10)


...

agrregate_evaluate results list length is  5
server round is  29
loss is  0.1628877967596054


DEBUG flwr 2023-09-01 00:05:20,864 | server.py:236 | fit_round 30 received 5 results and 0 failures
DEBUG flwr 2023-09-01 00:05:20,916 | server.py:173 | evaluate_round 30: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 30 aggregated_parameters...


DEBUG flwr 2023-09-01 00:05:21,710 | server.py:187 | evaluate_round 30 received 5 results and 0 failures
INFO flwr 2023-09-01 00:05:21,713 | server.py:153 | FL finished in 165.2082664000045
INFO flwr 2023-09-01 00:05:21,715 | app.py:225 | app_fit: losses_distributed [(1, 0.25283083319664), (2, 0.2535133957862854), (3, 0.25394898653030396), (4, 0.25337880849838257), (5, 0.2516672909259796), (6, 0.24831843376159668), (7, 0.2444283813238144), (8, 0.23920822143554688), (9, 0.23329900205135345), (10, 0.2277498096227646), (11, 0.22238850593566895), (12, 0.21630100905895233), (13, 0.20941436290740967), (14, 0.2051079124212265), (15, 0.19963374733924866), (16, 0.19395531713962555), (17, 0.19024166464805603), (18, 0.18459147214889526), (19, 0.1814517378807068), (20, 0.17778655886650085), (21, 0.17626971006393433), (22, 0.1725986748933792), (23, 0.16973111033439636), (24, 0.17027659714221954), (25, 0.16522088646888733), (26, 0.16657601296901703), (27, 0.1643616259098053), (28, 0.1633514910936355

agrregate_evaluate results list length is  5
server round is  30
loss is  0.16398300230503082
History (loss, distributed):
	round 1: 0.25283083319664
	round 2: 0.2535133957862854
	round 3: 0.25394898653030396
	round 4: 0.25337880849838257
	round 5: 0.2516672909259796
	round 6: 0.24831843376159668
	round 7: 0.2444283813238144
	round 8: 0.23920822143554688
	round 9: 0.23329900205135345
	round 10: 0.2277498096227646
	round 11: 0.22238850593566895
	round 12: 0.21630100905895233
	round 13: 0.20941436290740967
	round 14: 0.2051079124212265
	round 15: 0.19963374733924866
	round 16: 0.19395531713962555
	round 17: 0.19024166464805603
	round 18: 0.18459147214889526
	round 19: 0.1814517378807068
	round 20: 0.17778655886650085
	round 21: 0.17626971006393433
	round 22: 0.1725986748933792
	round 23: 0.16973111033439636
	round 24: 0.17027659714221954
	round 25: 0.16522088646888733
	round 26: 0.16657601296901703
	round 27: 0.1643616259098053
	round 28: 0.16335149109363556
	round 29: 0.1628877967596054

In [19]:
splitting = [0.6, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04]
do_fl(splitting, num_rounds)

Fold 1:
x_test: cpu
y_test: cpu
30


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Client 0 trainset shape torch.Size([134444, 10])
Client 1 trainset shape torch.Size([8908, 10])
Client 2 trainset shape torch.Size([8976, 10])
Client 3 trainset shape torch.Size([8900, 10])
Client 4 trainset shape torch.Size([9052, 10])
Client 5 trainset shape torch.Size([8910, 10])
Client 6 trainset shape torch.Size([8952, 10])
Client 7 trainset shape torch.Size([8938, 10])
Client 8 trainset shape torch.Size([8982, 10])
Client 9 trainset shape torch.Size([8950, 10])
FL started training on cpu
Number of available GPUs: 0


2023-09-01 00:05:39,101	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8271 
INFO flwr 2023-09-01 00:05:55,153 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1351100007.0, 'object_store_memory': 675550003.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-01 00:05:55,158 | server.py:89 | Initializing global parameters
INFO flwr 2023-09-01 00:05:55,160 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-09-01 00:06:12,455 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-09-01 00:06:12,459 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-09-01 00:06:12,461 | server.py:104 | FL starting
DEBUG flwr 2023-09-01 00:06:12,463 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-09-01 00:06:55,219 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-09-01 00:06:55,253 | fedavg

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  5
server round is  29
loss is  0.15587234497070312


DEBUG flwr 2023-09-01 00:11:42,921 | server.py:236 | fit_round 30 received 5 results and 0 failures
DEBUG flwr 2023-09-01 00:11:42,948 | server.py:173 | evaluate_round 30: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 30 aggregated_parameters...


DEBUG flwr 2023-09-01 00:11:43,316 | server.py:187 | evaluate_round 30 received 5 results and 0 failures
INFO flwr 2023-09-01 00:11:43,318 | server.py:153 | FL finished in 330.8548007000063
INFO flwr 2023-09-01 00:11:43,320 | app.py:225 | app_fit: losses_distributed [(1, 0.265170693397522), (2, 0.2644081115722656), (3, 0.2623937427997589), (4, 0.2597842216491699), (5, 0.25811341404914856), (6, 0.2562730610370636), (7, 0.21803508698940277), (8, 0.21398404240608215), (9, 0.1872166246175766), (10, 0.17134477198123932), (11, 0.1655968576669693), (12, 0.16567108035087585), (13, 0.15888701379299164), (14, 0.1639409363269806), (15, 0.1600913107395172), (16, 0.15779340267181396), (17, 0.15928326547145844), (18, 0.15971487760543823), (19, 0.16135862469673157), (20, 0.15543100237846375), (21, 0.15311220288276672), (22, 0.15635330975055695), (23, 0.15805166959762573), (24, 0.1596413403749466), (25, 0.16171088814735413), (26, 0.15769506990909576), (27, 0.15657919645309448), (28, 0.1547133475542068

agrregate_evaluate results list length is  5
server round is  30
loss is  0.15863412618637085
History (loss, distributed):
	round 1: 0.265170693397522
	round 2: 0.2644081115722656
	round 3: 0.2623937427997589
	round 4: 0.2597842216491699
	round 5: 0.25811341404914856
	round 6: 0.2562730610370636
	round 7: 0.21803508698940277
	round 8: 0.21398404240608215
	round 9: 0.1872166246175766
	round 10: 0.17134477198123932
	round 11: 0.1655968576669693
	round 12: 0.16567108035087585
	round 13: 0.15888701379299164
	round 14: 0.1639409363269806
	round 15: 0.1600913107395172
	round 16: 0.15779340267181396
	round 17: 0.15928326547145844
	round 18: 0.15971487760543823
	round 19: 0.16135862469673157
	round 20: 0.15543100237846375
	round 21: 0.15311220288276672
	round 22: 0.15635330975055695
	round 23: 0.15805166959762573
	round 24: 0.1596413403749466
	round 25: 0.16171088814735413
	round 26: 0.15769506990909576
	round 27: 0.15657919645309448
	round 28: 0.15471334755420685
	round 29: 0.1558723449707031

In [20]:
splitting = [0.8, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
do_fl(splitting, num_rounds)

Fold 1:
x_test: cpu
y_test: cpu
30


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_23376\475339861.py:32: UserWarning: To 

Client 0 trainset shape torch.Size([179330, 10])
Client 1 trainset shape torch.Size([4432, 10])
Client 2 trainset shape torch.Size([4500, 10])
Client 3 trainset shape torch.Size([4472, 10])
Client 4 trainset shape torch.Size([4480, 10])
Client 5 trainset shape torch.Size([4468, 10])
Client 6 trainset shape torch.Size([4470, 10])
Client 7 trainset shape torch.Size([4478, 10])
Client 8 trainset shape torch.Size([4476, 10])
Client 9 trainset shape torch.Size([4430, 10])
FL started training on cpu
Number of available GPUs: 0


2023-09-01 00:11:52,976	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8271 
INFO flwr 2023-09-01 00:12:01,742 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1629895067.0, 'object_store_memory': 814947532.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-01 00:12:01,745 | server.py:89 | Initializing global parameters
INFO flwr 2023-09-01 00:12:01,747 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-09-01 00:12:06,451 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-09-01 00:12:06,452 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-09-01 00:12:06,453 | server.py:104 | FL starting
DEBUG flwr 2023-09-01 00:12:06,454 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-09-01 00:12:37,855 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-09-01 00:12:37,886 | fedavg

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  5
server round is  29
loss is  0.15027034282684326


DEBUG flwr 2023-09-01 00:19:05,456 | server.py:236 | fit_round 30 received 5 results and 0 failures
DEBUG flwr 2023-09-01 00:19:05,517 | server.py:173 | evaluate_round 30: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 30 aggregated_parameters...


DEBUG flwr 2023-09-01 00:19:06,746 | server.py:187 | evaluate_round 30 received 5 results and 0 failures
INFO flwr 2023-09-01 00:19:06,750 | server.py:153 | FL finished in 420.29644460001145
INFO flwr 2023-09-01 00:19:06,754 | app.py:225 | app_fit: losses_distributed [(1, 0.22483320534229279), (2, 0.19219356775283813), (3, 0.16778017580509186), (4, 0.16737869381904602), (5, 0.1641659289598465), (6, 0.1614505499601364), (7, 0.15898959338665009), (8, 0.1568654477596283), (9, 0.15794648230075836), (10, 0.1523032784461975), (11, 0.15239271521568298), (12, 0.15139484405517578), (13, 0.1596277952194214), (14, 0.15414217114448547), (15, 0.15313319861888885), (16, 0.15172013640403748), (17, 0.15503646433353424), (18, 0.15554293990135193), (19, 0.1572270542383194), (20, 0.15729308128356934), (21, 0.15393811464309692), (22, 0.15680156648159027), (23, 0.1517280638217926), (24, 0.14946094155311584), (25, 0.14624756574630737), (26, 0.14640794694423676), (27, 0.15209032595157623), (28, 0.14953333139

agrregate_evaluate results list length is  5
server round is  30
loss is  0.16273850202560425
History (loss, distributed):
	round 1: 0.22483320534229279
	round 2: 0.19219356775283813
	round 3: 0.16778017580509186
	round 4: 0.16737869381904602
	round 5: 0.1641659289598465
	round 6: 0.1614505499601364
	round 7: 0.15898959338665009
	round 8: 0.1568654477596283
	round 9: 0.15794648230075836
	round 10: 0.1523032784461975
	round 11: 0.15239271521568298
	round 12: 0.15139484405517578
	round 13: 0.1596277952194214
	round 14: 0.15414217114448547
	round 15: 0.15313319861888885
	round 16: 0.15172013640403748
	round 17: 0.15503646433353424
	round 18: 0.15554293990135193
	round 19: 0.1572270542383194
	round 20: 0.15729308128356934
	round 21: 0.15393811464309692
	round 22: 0.15680156648159027
	round 23: 0.1517280638217926
	round 24: 0.14946094155311584
	round 25: 0.14624756574630737
	round 26: 0.14640794694423676
	round 27: 0.15209032595157623
	round 28: 0.14953333139419556
	round 29: 0.150270342826